In [ ]:
from queue import PriorityQueue

class No:
    def __init__(self, estado, no_pai=None, aresta=None, custo=0.0, heuristica=0.0):
        self.estado = estado            # Estado é uma tupla de tuplas (cada pilha é imutável)
        self.no_pai = no_pai            
        self.aresta = aresta            # Ação realizada para chegar neste nó (ex: (origem, destino, caixa))
        self.custo = custo              
        self.heuristica = heuristica    

    def __repr__(self):
        return str(self.estado)

    def __lt__(self, outro):
        # Critério de comparação: custo acumulado + heurística
        return (self.custo + self.heuristica) < (outro.custo + outro.heuristica)


In [8]:
# Função para recuperar a sequência de estados do nó inicial ao nó final
def no_caminho(no):
    caminho = [no.estado]
    while no.no_pai is not None:
        caminho.append(no.estado)
        no = no.no_pai
    caminho.reverse()
    return caminho

In [9]:
# Função para recuperar a sequência de ações (movimentos realizados)
def vertice_caminho(no):
    caminho = []
    while no.no_pai is not None:
        if no.aresta is not None:
            caminho.append(no.aresta)
        no = no.no_pai
    caminho.reverse()
    return caminho

In [ ]:
class Visitados:
    def __init__(self):
        self.visitados = set()

    def adicionar(self, no):
        self.visitados.add(no.estado)

    def foi_visitado(self, no):
        return no.estado in self.visitados

    def tamanho(self):
        return len(self.visitados)

In [ ]:
class FilaPrioridade:
    def __init__(self):
        self.fila = PriorityQueue()

    def push(self, valor, item):
        self.fila.put((valor, item))

    def pop(self):
        if self.esta_vazio():
            return None
        else:
            (_, no) = self.fila.get()
            return no

    def esta_vazio(self):
        return self.fila.empty()

In [ ]:
class Problema:
    def iniciar(self):
        raise NotImplementedError

    def imprimir(self, no):
        raise NotImplementedError

    def testar_objetivo(self, no):
        raise NotImplementedError

    def gerar_sucessores(self, no):
        raise NotImplementedError

    def custo(self, no, no_sucessor):
        raise NotImplementedError

    def heuristica(self, estado):
        raise NotImplementedError

In [ ]:
# empilhamento de caixas
class EmpilhaCaixas(Problema):
    def __init__(self):
        self.num_posicoes = 7
        # Estado inicial: cada posição é uma pilha (tupla) – caixas representadas por seus pesos (kg)
        # Exemplo: posição 0 com [10], 1 com [30], 2 vazia, 3 com [10], 4 com [40], 5 com [20], 6 com [30]
        self.estado_inicial = ((10,), (30,), (), (10,), (40,), (20,), (30,))
        self.total_caixas = 6
        self.limite_pilha = 3
        # Estado objetivo: as caixas devem estar em pilhas de no máximo 3, nas posições iniciais,
        # organizadas em ordem decrescente (do fundo para o topo).
        # Para 6 caixas, teremos 2 pilhas: a 1ª com (40, 30, 10) e a 2ª com (30, 20, 10).
        self.estado_objetivo = ((40, 30, 10), (30, 20, 10)) + ((),) * (self.num_posicoes - 2)

    def iniciar(self):
        no_raiz = No(self.estado_inicial, custo=0.0, heuristica=self.heuristica(self.estado_inicial))
        return no_raiz

    def imprimir(self, no):
        estado = no.estado
        s = ""
        for i, pilha in enumerate(estado):
            s += f"Posição {i}: {list(pilha)}\n"
        return s

    def testar_objetivo(self, no):
        estado = no.estado
        # Verifica o limite de caixas por pilha
        for pilha in estado:
            if len(pilha) > self.limite_pilha:
                return False
        # Verifica se as pilhas não vazias estão contíguas (nas casas iniciais, sem lacunas)
        found_empty = False
        non_empty_count = 0
        for pilha in estado:
            if len(pilha) == 0:
                found_empty = True
            else:
                if found_empty:
                    return False
                non_empty_count += 1
                # Verifica se a pilha está ordenada de forma decrescente (do fundo para o topo)
                for i in range(len(pilha) - 1):
                    if not (pilha[i] > pilha[i + 1]):
                        return False
        # Para 6 caixas com limite de 3, espera-se 2 pilhas não vazias
        if non_empty_count != ((self.total_caixas + self.limite_pilha - 1) // self.limite_pilha):
            return False
        # Verifica se o estado é exatamente o desejado.
        return estado == self.estado_objetivo

    def gerar_sucessores(self, no):
        sucessores = []
        estado_atual = no.estado
        # Para cada posição de origem que contenha ao menos uma caixa...
        for i in range(self.num_posicoes):
            if len(estado_atual[i]) == 0:
                continue
            caixa = estado_atual[i][-1]  # pega a caixa do topo
            # Tenta mover para cada outra posição
            for j in range(self.num_posicoes):
                if i == j:
                    continue
                # Verifica se a posição destino não está cheia
                if len(estado_atual[j]) >= self.limite_pilha:
                    continue
                # Se a pilha destino não está vazia, só permite se o topo for maior que a caixa (para manter ordem decrescente)
                if len(estado_atual[j]) > 0 and estado_atual[j][-1] <= caixa:
                    continue
                # Gera novo estado: remove a caixa de i e adiciona em j
                novo_estado = [list(p) for p in estado_atual]
                novo_estado[i].pop()
                novo_estado[j].append(caixa)
                novo_estado = tuple(tuple(p) for p in novo_estado)
                # Define a ação realizada (origem, destino, caixa)
                acao = (i, j, caixa)
                # Calcula o custo do movimento
                custo_mov = self.calcular_custo(i, j, caixa)
                novo_no = No(novo_estado, no, acao, no.custo + custo_mov, self.heuristica(novo_estado))
                sucessores.append(novo_no)
        return sucessores

    def calcular_custo(self, origem, destino, caixa):
        distancia = abs(origem - destino)
        # Se mover 1 casa: custo = 1; se mover 2 ou mais: custo = distância * 0.75
        custo_mov = distancia * (1.0 if distancia == 1 else 0.75)
        custo_peso = caixa / 10.0  # A cada 10kg, aumenta 1 de custo
        return custo_mov + custo_peso

    def custo(self, no, no_sucessor):
        return no_sucessor.custo - no.custo

    def heuristica(self, estado):
        h = 0.0
        # Primeiro: penaliza caixas que estão fora das pilhas alvo (índices 0 e 1)
        # Para cada caixa em posições 2 até (num_posicoes-1):
        for i in range(2, self.num_posicoes):
            for box in estado[i]:
                # Custo mínimo para mover a caixa para a posição mais próxima dentre 0 e 1:
                d = min(abs(i - 0), abs(i - 1))
                if d == 1:
                    custo_min = 1 + (box / 10.0)
                else:
                    custo_min = d * 0.75 + (box / 10.0)
                h += custo_min

        # Segundo: para as pilhas alvo (posições 0 e 1), penaliza diferenças em relação ao alvo
        target0 = (40, 30, 10)
        target1 = (30, 20, 10)
        for idx, target in zip([0, 1], [target0, target1]):
            stack = estado[idx]
            # Para cada posição esperada na pilha alvo:
            for pos in range(len(target)):
                # Se a posição não existe ou o valor não confere, adicione um custo mínimo
                if pos >= len(stack) or stack[pos] != target[pos]:
                    # Penaliza com custo mínimo de um movimento para inserir ou corrigir a caixa
                    h += 1 + (target[pos] / 10.0)
        return h

In [ ]:
# Dijkstra (A* com h = 0)
def dijkstra(problema):
    no_inicial = problema.iniciar()
    fila = FilaPrioridade()
    # Usa somente o custo acumulado como chave na fila
    fila.push(no_inicial.custo, no_inicial)
    visitados = Visitados()
    visitados.adicionar(no_inicial)

    while not fila.esta_vazio():
        no_atual = fila.pop()
        if problema.testar_objetivo(no_atual):
            return (visitados.tamanho(), no_atual)
        for sucessor in problema.gerar_sucessores(no_atual):
            if not visitados.foi_visitado(sucessor):
                visitados.adicionar(sucessor)
                fila.push(sucessor.custo, sucessor)
    return (visitados.tamanho(), None)

In [ ]:
# A* (utiliza custo acumulado + heurística)
def a_estrela(problema):
    no_inicial = problema.iniciar()
    fila = FilaPrioridade()
    fila.push(no_inicial.custo + no_inicial.heuristica, no_inicial)
    visitados = Visitados()
    visitados.adicionar(no_inicial)

    while not fila.esta_vazio():
        no_atual = fila.pop()
        if problema.testar_objetivo(no_atual):
            return (visitados.tamanho(), no_atual)
        for sucessor in problema.gerar_sucessores(no_atual):
            if not visitados.foi_visitado(sucessor):
                visitados.adicionar(sucessor)
                fila.push(sucessor.custo + sucessor.heuristica, sucessor)
    return (visitados.tamanho(), None)


In [ ]:
# ganancioso (Greedy) – prioriza somente a heurística
def greedy(problema):
    no_inicial = problema.iniciar()
    fila = FilaPrioridade()
    fila.push(no_inicial.heuristica, no_inicial)
    visitados = Visitados()
    visitados.adicionar(no_inicial)

    while not fila.esta_vazio():
        no_atual = fila.pop()
        if problema.testar_objetivo(no_atual):
            return (visitados.tamanho(), no_atual)
        for sucessor in problema.gerar_sucessores(no_atual):
            if not visitados.foi_visitado(sucessor):
                visitados.adicionar(sucessor)
                fila.push(sucessor.heuristica, sucessor)
    return (visitados.tamanho(), None)


In [ ]:
# executa os três algoritmos e mostra seus resultados
if __name__ == "__main__":
    problema = EmpilhaCaixas()

    print("=== Busca Dijkstra ===")
    qtd_visitados_d, no_solucao_d = dijkstra(problema)
    if no_solucao_d is None:
        print("Não houve solução para o problema.")
    else:
        print("Solução encontrada:")
        print("Ações (movimentos realizados):")
        print(vertice_caminho(no_solucao_d))
        print("\nCaminho de estados:")
        for estado in no_caminho(no_solucao_d):
            print(estado)
    print(f"\nEstados visitados: {qtd_visitados_d}")
    print("\nEstado inicial:")
    print(problema.imprimir(No(problema.estado_inicial)))
    
    print("\n=== Busca A* ===")
    qtd_visitados_a, no_solucao_a = a_estrela(problema)
    if no_solucao_a is None:
        print("Não houve solução para o problema.")
    else:
        print("Solução encontrada:")
        print("Ações (movimentos realizados):")
        print(vertice_caminho(no_solucao_a))
        print("\nCaminho de estados:")
        for estado in no_caminho(no_solucao_a):
            print(estado)
    print(f"\nEstados visitados: {qtd_visitados_a}")
    
    print("\n=== Busca Gananciosa (Greedy) ===")
    qtd_visitados_g, no_solucao_g = greedy(problema)
    if no_solucao_g is None:
        print("Não houve solução para o problema.")
    else:
        print("Solução encontrada:")
        print("Ações (movimentos realizados):")
        print(vertice_caminho(no_solucao_g))
        print("\nCaminho de estados:")
        for estado in no_caminho(no_solucao_g):
            print(estado)
    print(f"\nEstados visitados: {qtd_visitados_g}")

=== Busca Dijkstra ===
Solução encontrada:
Ações (movimentos realizados):
[(5, 1, 20), (0, 1, 10), (4, 0, 40), (6, 0, 30), (3, 0, 10)]

Caminho de estados:
((10,), (30, 20), (), (10,), (40,), (), (30,))
((), (30, 20, 10), (), (10,), (40,), (), (30,))
((40,), (30, 20, 10), (), (10,), (), (), (30,))
((40, 30), (30, 20, 10), (), (10,), (), (), ())
((40, 30, 10), (30, 20, 10), (), (), (), (), ())
((40, 30, 10), (30, 20, 10), (), (), (), (), ())

Estados visitados: 20541

Estado inicial:
Posição 0: [10]
Posição 1: [30]
Posição 2: []
Posição 3: [10]
Posição 4: [40]
Posição 5: [20]
Posição 6: [30]


=== Busca A* ===
Solução encontrada:
Ações (movimentos realizados):
[(5, 1, 20), (0, 1, 10), (4, 0, 40), (6, 0, 30), (3, 0, 10)]

Caminho de estados:
((10,), (30, 20), (), (10,), (40,), (), (30,))
((), (30, 20, 10), (), (10,), (40,), (), (30,))
((40,), (30, 20, 10), (), (10,), (), (), (30,))
((40, 30), (30, 20, 10), (), (10,), (), (), ())
((40, 30, 10), (30, 20, 10), (), (), (), (), ())
((40, 30, 